Read the following data set: https://archive.ics.uci.edu/ml/machine-learning-databases/adult

In [1]:
# DownLoading the adult.data,adult.names, and adult.test data writing into respective files 
# and storing into current working folder using os and requests python libraries
import os
import requests
ADULT_DATASET = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names",
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
)

# Writing a function download_data to download the adult folder related all the data into current working folder
def download_data(path='.', urls=ADULT_DATASET):
    #Checks if path does not exists, then create respective directory
    if not os.path.exists(path):
        os.mkdir(path)
        
    #tuple has been passed and running a for loop for reading data from url using requests and storing into physical files.
    for url in urls:
        response = requests.get(url)
        name = os.path.basename(url)
        with open(os.path.join(path, name), 'w') as f:
            f.write(response.text)
# End of download_data function
            
#Calling download_data function    
download_data()

# Listing files in the directory
os.listdir()

['.ipynb_checkpoints',
 'adult.data',
 'adult.names',
 'adult.test',
 'Assignment_7.11439.ipynb']

Rename the columns as per the description from this file: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

In [2]:
import pandas as pd
names = ['age','workclass','fnlwgt','education','education-num',
        'marital-status','occupation','relationship','race',
        'sex','capital-gain','capital-loss','hours-per-week',
        'native-country','income',]
#sep=r'\s*,\s* considers white space along with comma separator
#na_values="?" replaces ? with NaN values for further cleaning and churning of data 
data = pd.read_csv('adult.data', names=names,sep=r'\s*,\s*',engine='python',na_values="?")
data.info()
#data =data.dropna()
#data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         30725 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        30718 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    31978 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Create a sql db from adult dataset and name it sqladb

In [3]:
import sqlite3 as db

#Create and/or connects to adult.db sqlite3 database file
sqladb = db.connect("adult.db")

#Inserting dataframe data into adult_tbl
data.to_sql('adult_tbl', sqladb, if_exists='replace', index=False)

#Creating database cursor for further querying the data
c=sqladb.cursor()

Task 1.1 Select 10 records from the adult sqladb

In [4]:
query = """ SELECT * FROM adult_tbl limit 10 """    #other way c.execute("SELECT * FROM adult_tbl limit 10")
pd.read_sql_query(query,sqladb)                     #other way #print(c.fetchall())

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


Task 1.2 Show me the average hours per week of all men who are working in private sector

In [5]:
c.execute("""SELECT sex,workclass, AVG("hours-per-week") AverageHoursPerWeek FROM adult_tbl 
             where sex='Male' and workclass='Private'  group by sex,workclass  """)
print(c.fetchone())

('Male', 'Private', 42.22122591006424)


In [6]:
query = """ SELECT sex,workclass, avg("hours-per-week") AverageHoursPerWeek FROM adult_tbl 
            where sex='Male' and workclass='Private'  group by sex,workclass   """    
pd.read_sql_query(query,sqladb)                     

,sex,workclass,AverageHoursPerWeek
0,Male,Private,42.221226


Task 1.3 Show me the frequency table for education, occupation and relationship, separately

In [8]:
df = pd.read_sql_query("""SELECT  education, count(education) frequency from adult_tbl 
                       group by education order by frequency desc """,sqladb)
display(df)

,education,frequency
0,HS-grad,10501
1,Some-college,7291
2,Bachelors,5355
3,Masters,1723
4,Assoc-voc,1382
5,11th,1175
6,Assoc-acdm,1067
7,10th,933
8,7th-8th,646
9,Prof-school,576


In [9]:
df1 = pd.read_sql_query("""SELECT  occupation, count(occupation) frequency from adult_tbl
                        group by occupation order by frequency desc """,sqladb)
display(df1)

,occupation,frequency
0,Prof-specialty,4140
1,Craft-repair,4099
2,Exec-managerial,4066
3,Adm-clerical,3770
4,Sales,3650
5,Other-service,3295
6,Machine-op-inspct,2002
7,Transport-moving,1597
8,Handlers-cleaners,1370
9,Farming-fishing,994


In [10]:
df2 = pd.read_sql_query("""SELECT  relationship, count(relationship) frequency from adult_tbl
                        group by relationship order by frequency desc """,sqladb)
display(df2)

,relationship,frequency
0,Husband,13193
1,Not-in-family,8305
2,Own-child,5068
3,Unmarried,3446
4,Wife,1568
5,Other-relative,981


Task 1.4 Are there any people who are married, working in private sector and having a masters degree

In [11]:
#education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
#workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
#marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
df2 = pd.read_sql_query("""SELECT count(*) as People FROM adult_tbl where 1=1
                            and "marital-status" like 'Married-%'
                            and workclass='Private' 
                            and education='Masters' """,sqladb)
display(df2)

,People
0,540


Task 1.5 What is the average, minimum and maximum age group for people working in different sectors

In [13]:
query = """ SELECT workclass, avg(age) mean,min(age) min,max(age) max from adult_tbl group by workclass """
pd.read_sql_query(query,sqladb)

,workclass,mean,min,max
0,None,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


Task 1.6. Calculate age distribution by country

In [14]:
query = """ SELECT "native-country" as Country, Round(Round((count(age)*100),5)/Round((select count(age) 
            from adult_tbl),5),5) as "Age Distribution"  FROM adult_tbl 
            group by "native-country" order by count(age) desc  """
pd.read_sql_query(query,sqladb)

,Country,Age Distribution
0,United-States,89.58570
1,Mexico,1.97476
2,None,1.79049
3,Philippines,0.60809
4,Germany,0.42075
5,Canada,0.37161
6,Puerto-Rico,0.35011
7,El-Salvador,0.32554
8,India,0.30712
9,Cuba,0.29176


Task 1.7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

In [20]:
#Net Capital Gain= Sum(Capital Gain)-Sum(Capital Loss)
query = """ SELECT sum("capital-gain") AS "Capital-Gain",sum("capital-loss") AS "Capital-Loss", (sum("capital-gain")-sum("capital-loss")) 
                    "Net-Capital-Gain" FROM adult_tbl  """
pd.read_sql_query(query,sqladb)

,Capital-Gain,Capital-Loss,Net-Capital-Gain
0,35089324,2842700,32246624


In [21]:
#Each Row Specific Net Capital Gain Calculation
#Net Capital Gain= (Capital Gain)-(Capital Loss)  
query = """ SELECT "capital-gain","capital-loss", ("capital-gain"-"capital-loss") 
                    "Net-Capital-Gain" FROM adult_tbl where "capital-gain">0 or "capital-loss">0  """
pd.read_sql_query(query,sqladb)

,capital-gain,capital-loss,Net-Capital-Gain
0,2174,0,2174
1,14084,0,14084
2,5178,0,5178
3,0,2042,-2042
4,0,1408,-1408
5,0,1902,-1902
6,5013,0,5013
7,2407,0,2407
8,14344,0,14344
9,0,1573,-1573


Task 2: Read the following data set: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Task 2.1. Create an sqlalchemy engine using a sample from the data set

In [22]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
db_uri = "sqlite:///adult.db"
engine = create_engine(db_uri)

# Create a MetaData instance
metadata = MetaData()
print(metadata.tables)

# reflect db schema to MetaData
metadata.reflect(bind=engine)
print(metadata.tables)

# Get Table
ex_table = metadata.tables['adult_tbl']
print(ex_table)

# create session
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

immutabledict({})
immutabledict({'adult_tbl': Table('adult_tbl', MetaData(bind=None), Column('age', INTEGER(), table=<adult_tbl>), Column('workclass', TEXT(), table=<adult_tbl>), Column('fnlwgt', INTEGER(), table=<adult_tbl>), Column('education', TEXT(), table=<adult_tbl>), Column('education-num', INTEGER(), table=<adult_tbl>), Column('marital-status', TEXT(), table=<adult_tbl>), Column('occupation', TEXT(), table=<adult_tbl>), Column('relationship', TEXT(), table=<adult_tbl>), Column('race', TEXT(), table=<adult_tbl>), Column('sex', TEXT(), table=<adult_tbl>), Column('capital-gain', INTEGER(), table=<adult_tbl>), Column('capital-loss', INTEGER(), table=<adult_tbl>), Column('hours-per-week', INTEGER(), table=<adult_tbl>), Column('native-country', TEXT(), table=<adult_tbl>), Column('income', TEXT(), table=<adult_tbl>), schema=None)})
adult_tbl


Task 2.2 Write two basic update queries

In [23]:
from sqlalchemy.sql import select
from sqlalchemy import update
connection = engine.connect()

#Update Query 1 where age of 39 should be changed to 40
u = update(ex_table, ex_table.c.age=='39')
connection.execute(u, age="40")

#Update Query 2 where relationship wife should be changed to wifi
u = update(ex_table, ex_table.c.relationship=='Wife')
connection.execute(u, relationship='Wifi')

s = select([ex_table])
result=connection.execute(s)
display(result.fetchmany(10))
connection.close()

[(40, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States', '<=50K'),
 (50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 13, 'United-States', '<=50K'),
 (38, 'Private', 215646, 'HS-grad', 9, 'Divorced', 'Handlers-cleaners', 'Not-in-family', 'White', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (53, 'Private', 234721, '11th', 7, 'Married-civ-spouse', 'Handlers-cleaners', 'Husband', 'Black', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (28, 'Private', 338409, 'Bachelors', 13, 'Married-civ-spouse', 'Prof-specialty', 'Wifi', 'Black', 'Female', 0, 0, 40, 'Cuba', '<=50K'),
 (37, 'Private', 284582, 'Masters', 14, 'Married-civ-spouse', 'Exec-managerial', 'Wifi', 'White', 'Female', 0, 0, 40, 'United-States', '<=50K'),
 (49, 'Private', 160187, '9th', 5, 'Married-spouse-absent', 'Other-service', 'Not-in-family', 'Black', 'Female', 0, 0, 16, 'J

Task 2.3. Write two delete queries

In [28]:
from sqlalchemy import delete
connection = engine.connect()
#Delete Query to delete records with age is 40
d = delete(ex_table, ex_table.c.age==40)
connection.execute(d)

#Delete Query to delete records with relationship is Wifi
d = delete(ex_table, ex_table.c.relationship=='Wifi')
connection.execute(d)

s = select([ex_table])
result = connection.execute(s)
display(result.fetchmany(10))
connection.close()

[(50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 13, 'United-States', '<=50K'),
 (38, 'Private', 215646, 'HS-grad', 9, 'Divorced', 'Handlers-cleaners', 'Not-in-family', 'White', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (53, 'Private', 234721, '11th', 7, 'Married-civ-spouse', 'Handlers-cleaners', 'Husband', 'Black', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (49, 'Private', 160187, '9th', 5, 'Married-spouse-absent', 'Other-service', 'Not-in-family', 'Black', 'Female', 0, 0, 16, 'Jamaica', '<=50K'),
 (52, 'Self-emp-not-inc', 209642, 'HS-grad', 9, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 45, 'United-States', '>50K'),
 (31, 'Private', 45781, 'Masters', 14, 'Never-married', 'Prof-specialty', 'Not-in-family', 'White', 'Female', 14084, 0, 50, 'United-States', '>50K'),
 (42, 'Private', 159449, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male',

Task 2.4 Write two filter queries

In [29]:
from sqlalchemy.sql import select
connection = engine.connect()
#Select Query 1 for selecting records where age is greater than 40
s = select([ex_table], ex_table.c.age>=45)
result = connection.execute(s)
display(result.fetchmany(10))
connection.close()

[(50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 13, 'United-States', '<=50K'),
 (53, 'Private', 234721, '11th', 7, 'Married-civ-spouse', 'Handlers-cleaners', 'Husband', 'Black', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (49, 'Private', 160187, '9th', 5, 'Married-spouse-absent', 'Other-service', 'Not-in-family', 'Black', 'Female', 0, 0, 16, 'Jamaica', '<=50K'),
 (52, 'Self-emp-not-inc', 209642, 'HS-grad', 9, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 45, 'United-States', '>50K'),
 (54, 'Private', 302146, 'HS-grad', 9, 'Separated', 'Other-service', 'Unmarried', 'Black', 'Female', 0, 0, 20, 'United-States', '<=50K'),
 (59, 'Private', 109015, 'HS-grad', 9, 'Divorced', 'Tech-support', 'Unmarried', 'White', 'Female', 0, 0, 40, 'United-States', '<=50K'),
 (56, 'Local-gov', 216851, 'Bachelors', 13, 'Married-civ-spouse', 'Tech-support', 'Husband', 'White', 'Male', 0, 0, 40, 'United-S

In [33]:
#Select Query 2 for selecting records where relationship is Husband
connection = engine.connect()
s = select([ex_table], ex_table.c.relationship=='Husband')
result = connection.execute(s)
display(result.fetchmany(10))
connection.close()

[(50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 13, 'United-States', '<=50K'),
 (53, 'Private', 234721, '11th', 7, 'Married-civ-spouse', 'Handlers-cleaners', 'Husband', 'Black', 'Male', 0, 0, 40, 'United-States', '<=50K'),
 (52, 'Self-emp-not-inc', 209642, 'HS-grad', 9, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 45, 'United-States', '>50K'),
 (42, 'Private', 159449, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 5178, 0, 40, 'United-States', '>50K'),
 (37, 'Private', 280464, 'Some-college', 10, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'Black', 'Male', 0, 0, 80, 'United-States', '>50K'),
 (30, 'State-gov', 141297, 'Bachelors', 13, 'Married-civ-spouse', 'Prof-specialty', 'Husband', 'Asian-Pac-Islander', 'Male', 0, 0, 40, 'India', '>50K'),
 (34, 'Private', 245487, '7th-8th', 4, 'Married-civ-spouse', 'Transport-moving', 'Husband', 'A

Task 2.5 Write two function queries

In [34]:
import sqlite3

#function 1 connects or create database file
def connect(sqlite_file):
    """ Make connection to an SQLite database file """
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    return conn, c

#function 2 close connection with SQLite database
def close(conn):
    """ Commit changes and close connection to the database """
    conn.close()

#function 3 counts total rows
def total_rows(cursor, table_name, print_out=False):
    """ Returns the total number of rows in the database """
    cursor.execute('SELECT COUNT(*) FROM {}'.format(table_name))
    count = cursor.fetchall()
    if print_out:
        print('\nTotal rows: {}'.format(count[0][0]))
    return count[0][0]

#function 4 gives table columns information
def table_col_info(cursor, table_name, print_out=False):
    """ Returns a list of tuples with column informations:
    (id, name, type, notnull, default_value, primary_key)
    """
    cursor.execute('PRAGMA TABLE_INFO({})'.format(table_name))
    info = cursor.fetchall()
    if print_out:
        print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
        for col in info:
            print(col)
    return info

if __name__ == '__main__':
    sqlite_file = 'adult.db'
    table_name = 'adult_tbl'
    conn, c = connect(sqlite_file)                     #function 1  
    total_rows(c, table_name, print_out=True)          #function 2
    table_col_info(c, table_name, print_out=True)      #function 3
    close(conn)                                        #function 4


Total rows: 29499

Column Info:
ID, Name, Type, NotNull, DefaultVal, PrimaryKey
(0, 'age', 'INTEGER', 0, None, 0)
(1, 'workclass', 'TEXT', 0, None, 0)
(2, 'fnlwgt', 'INTEGER', 0, None, 0)
(3, 'education', 'TEXT', 0, None, 0)
(4, 'education-num', 'INTEGER', 0, None, 0)
(5, 'marital-status', 'TEXT', 0, None, 0)
(6, 'occupation', 'TEXT', 0, None, 0)
(7, 'relationship', 'TEXT', 0, None, 0)
(8, 'race', 'TEXT', 0, None, 0)
(9, 'sex', 'TEXT', 0, None, 0)
(10, 'capital-gain', 'INTEGER', 0, None, 0)
(11, 'capital-loss', 'INTEGER', 0, None, 0)
(12, 'hours-per-week', 'INTEGER', 0, None, 0)
(13, 'native-country', 'TEXT', 0, None, 0)
(14, 'income', 'TEXT', 0, None, 0)
